At a given snapshot, get all orbit data for each pair in pair catalog

This notebook will pull and save orbits with data:

- (float) Redshift 
- (flaot) Scale
- (int) GroupNum
- (int) SubID1
- (int) SubID2
- (float) Sub1StellarMass
- (float) Sub2StellarMass
- (obj) Group Flag vs. z
- (obj) Merge Flag vs. z
- (int) Merger snapshot 
- (float) Merger redshift 
- (float) Starting Redshift (FOF Group Infall)
- (int) Starting Snapshot (FOF Group Infall)
- (obj) Pos1 vs. z 
- (obj) Pos2 vs. z
- (obj) Sep vs. z
- (obj) Vel1 vs. z
- (obj) Vel2 vs. z
- (obj) RelVel vs. z
- (float) Separation before merger
- (int) Pairkey (maybe the SubhaloID 1+2 at z=4)


In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib
from astropy.cosmology import FlatLambdaCDM

from utils_orbs.orb_paths import SetupPaths
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector


paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [2]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

## Make the time, redshift, scale, snapshot conversion file


First, make the simulation data arrays, including:
snapshot, time, redshift, and scale

Will use this for all conversions in the future between redshift and snapshot number, etc.


In [4]:
if os.path.exists(f"{paths.path_data}collection/misc/snapshot_data.hdf5"):
    print ("file already exists")
    
else:    
    ## import file with cosmological parameters for TNG-hydro
    f = h5py.File(f"{paths.path_data}collection/misc/simulation.hdf5")

    ## initialize cosmology
    H0 = 100*f['Header'].attrs['HubbleParam']
    Omega0 = f['Header'].attrs['Omega0']
    OmegaB = f['Header'].attrs['OmegaBaryon']
    OmegaL = f['Header'].attrs['OmegaLambda']
    cosmo = FlatLambdaCDM(H0=67.74, Om0=Omega0, Ob0=OmegaB)

    ## initialize data structure
    data_dict = {}
    array_names = ["Snapshot","Redshift","Scale","Lookback Time","Age"]
    for arr in array_names:
        data_dict[arr] = []

    ## create data structure for time, redshift, scale, etc. as fxn of snapshot
    for i in range(100):
        snapshot = i
        redshift = f['Snapshots'][f'{i}']['Header'].attrs['Redshift']
        scale = 1 / (1 + redshift)
        lookback = cosmo.lookback_time(redshift).value
        age = cosmo.age(redshift).value

        data_dict['Snapshot'].append(snapshot)
        data_dict['Redshift'].append(redshift)
        data_dict['Scale'].append(scale)
        data_dict['Lookback Time'].append(lookback)
        data_dict['Age'].append(age)

    ## clsoe data file     
    f.close()

    ## store time data into hdf5 file ~

    f = h5py.File(f"{paths.path_data}collection/misc/snapshot_data.hdf5", 'w')

    info_dict = {"Snapshot":"Snapshot number",
                 "Redshift":"Redshift at snapshot",
                 "Scale":"Scale at snapshot",
                 "Lookback Time":"Time that has elapsed from snapshot to z=0 (in Gyr)",
                 "Age":"Age of the Universe (in Gyr)"}

    for key, val in data_dict.items():
        val = np.array(val)
        dset = f.create_dataset(f'/{key}', 
                                shape=val.shape,
                                dtype=val.dtype)
        dset.attrs[key] = info_dict[key]
        dset[:] = val

    f.close()    
    
    print("snapshot information file created")

snapshot information file created
